In [ ]:
import numpy as np;import pandas as pd;import os,sys;
from sklearn.model_selection import train_test_split,KFold
from collections import Counter
from utils import read_data
from sklearn.utils import shuffle;
import torch;from torch import nn, Tensor;import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset,DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE=128

types="O"
rna_type="pirna"
random_seed=123
def seq2kmer(seq):
    k=3
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

all_data=read_data("./dataset/set/"+types+'/'+rna_type+'-short')

train_data, val_data = train_test_split(all_data, test_size=0.2, random_state=random_seed)    
def get_bert_data():
    if os.path.exists("./dataset/bert/"+types+'_'+rna_type):
        print("目录已存在")
    else:
        os.mkdir("./dataset/bert/"+types+'_'+rna_type)
    df=train_data.copy(deep=True)
    df["sequence"]=df["sequence"].apply(seq2kmer)
    df.to_csv("./dataset/bert/"+types+'_'+rna_type+"/train.tsv",sep="\t",index=None)
    df=val_data.copy(deep=True)
    df["sequence"]=df["sequence"].apply(seq2kmer)
    df.to_csv("./dataset/bert/"+types+'_'+rna_type+"/dev.tsv",sep="\t",index=None)
get_bert_data()
eph=5
!python ./DNABERT/examples/run_finetune.py   --early_stop 10 --model_type dna  --tokenizer_name=dna3  --model_name_or_path ./DNABERT/3-new-12w-0  --task_name dnaprom  --do_train  --do_eval  --data_dir ./dataset/bert/{types}_{rna_type} --max_seq_length 101  --per_gpu_eval_batch_size=32    --per_gpu_train_batch_size=32   --learning_rate 3e-5  --num_train_epochs {str(eph)}  --output_dir ./results/models/bert/{types}_{rna_type}  --evaluate_during_training  --logging_steps 100  --save_steps 100  --warmup_percent 0.1  --hidden_dropout_prob 0.1  --overwrite_output  --weight_decay 0.01  --n_process 40  --fp16
